#### 2-1. Nate 뉴스기사 제목 스크래핑하기
https://news.nate.com/recent?mid=n0100   
최신뉴스, 정치 , 경제, 사회, 세계, IT/과학 6개의 섹션의 뉴스를 출력하는 함수를 생성하여 스크래핑 하기


In [16]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin  # url 합치는 도구
from IPython.display import Image, display # 이미지를 한 번에 보여주는 도구

In [2]:
section_dict = {'최신뉴스': 'n0100','정치': 'n0200',
                '경제': 'n0300','사회': 'n0400',
                '세계': 'n0500','IT/과학': 'n0600'}

In [14]:
def nate_news(section_name):
    sid = section_dict.get(section_name, 'IT/과학')

    url = f'https://news.nate.com/section?mid={sid}'
    print(f"====== [{section_name}] 뉴스 스크래핑 시작 : {url} ======")

    req_header = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/144.0.0.0 Safari/537.3'
    }

    res = requests.get(url, headers=req_header)
    res.encoding = 'utf-8' # res 선언 밑에다가 

    if res.ok:
        soup = BeautifulSoup(res.text, 'html.parser')
        articles = soup.select("div.mduSubjectList div.mduSubject")

        for article in articles:
            img_tag = article.select_one("span.tb img") or article.select_one("a.pic img")

            if img_tag:
                img_src = img_tag['src']
                # [핵심 2] 불완전한 주소(//...)를 완전한 주소(https://...)로 변환
                full_img_url = urljoin(url, img_src)
                
                # [핵심 3] 이미지 화면 출력
                print("[이미지]")
                display(Image(url=full_img_url, width=100)) # 크기 100px로 조절
            else:
                print(">> (이미지 없음)")

            title_tag = article.select_one("strong.tit") or article.select_one("h4.tit")
            link_tag = article.select_one("a") # 기사 전체를 감싸는 a태그
            
            if title_tag and link_tag:
                title = title_tag.text.strip()
                link = urljoin(url, link_tag['href']) # 링크도 안전하게 urljoin
                
                print(f"[제목] {title}")
                print(f"[링크] {link}")
                print("-" * 50)
    else:
        print(f"요청 실패! 상태 코드: {res.status_code}")




In [15]:
nate_news('정치')

====== [정치] 뉴스 스크래핑 시작 : https://news.nate.com/section?mid=n0200 ======


#### 2-2. 하나의 네이버 웹툰과 1개의 회차에 대한 Image 다운로드 하기


In [17]:
import os

In [20]:

def download_one_episode(title,no,url):

    req_header = {
        'referer': url,
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'
    }
    res = requests.get(url, headers=req_header)

    if res.ok:
        # 이미지 url 추출
        soup = BeautifulSoup(res.text, 'html.parser')
        img_tags = soup.select("img[src*='IMAG01']") # img 태그의 src 속성에서 'IMG01' 포함한 모든 태그 추출


        #리스트 컴프리헨션
        img_url_list = [img_tag['src'] for img_tag in img_tags]  
        # img_url_list2 에 모든 이미지 링크가 배열됨

        dir_path = os.path.join('img', title, str(no))
        if not os.path.exists(dir_path): 
            os.makedirs(dir_path)

        for img_url in img_url_list:
            res = requests.get(img_url, headers=req_header)  #header를 설정 안하면 403(권한 없음이 뜬다)
            img_data = res.content

            # url에서 파일명만 추출하기
            # img/xxxIMG01.jpg
            file_path = os.path.join(dir_path, os.path.basename(img_url))

            # binary 데이터를 file에 write 하기
            with open(file_path,'wb') as file:
                print(f'Writing to {file_path}({len(img_data):,}bytes)')
                file.write(img_data)

    else:
        print(f'Error Code = {res.status_code}')




In [21]:
# 실행 테스트
download_one_episode('엄마를 만나러 가는 길', 1, 'https://comic.naver.com/webtoon/detail?titleId=834369&no=1&week=finish')

Writing to img\엄마를 만나러 가는 길\1\20241125101030_b19e80838a6e85b8aca65b3b02d47b28_IMAG01_1.jpg(31,242bytes)
Writing to img\엄마를 만나러 가는 길\1\20241125101030_b19e80838a6e85b8aca65b3b02d47b28_IMAG01_2.jpg(73,417bytes)
Writing to img\엄마를 만나러 가는 길\1\20241125101030_b19e80838a6e85b8aca65b3b02d47b28_IMAG01_3.jpg(101,481bytes)
Writing to img\엄마를 만나러 가는 길\1\20241125101030_b19e80838a6e85b8aca65b3b02d47b28_IMAG01_4.jpg(81,404bytes)
Writing to img\엄마를 만나러 가는 길\1\20241125101030_b19e80838a6e85b8aca65b3b02d47b28_IMAG01_5.jpg(172,033bytes)
Writing to img\엄마를 만나러 가는 길\1\20241125101030_b19e80838a6e85b8aca65b3b02d47b28_IMAG01_6.jpg(111,113bytes)
Writing to img\엄마를 만나러 가는 길\1\20241125101030_b19e80838a6e85b8aca65b3b02d47b28_IMAG01_7.jpg(50,877bytes)
Writing to img\엄마를 만나러 가는 길\1\20241125101030_b19e80838a6e85b8aca65b3b02d47b28_IMAG01_8.jpg(172,144bytes)
Writing to img\엄마를 만나러 가는 길\1\20241125101030_b19e80838a6e85b8aca65b3b02d47b28_IMAG01_9.jpg(96,121bytes)
Writing to img\엄마를 만나러 가는 길\1\20241125101030_b19e80838a6e85b